In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from scipy import ndimage
from PIL import Image
import glob
import imutils

import os

In [3]:
# Load ToF image
images_normalized=[]

#Gloria path
#for filename in glob.glob('/content/drive/MyDrive/PerceptionAI/data/depths/*.png'):

#Julie Path
for filename in glob.glob('./depths/*.png'):
    #print("here")
    frame = cv2.imread(filename)

    # Normalize the image
    img_normalized = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    images_normalized.append(img_normalized)
    # Display the normalized image
    #cv2.imshow("image normalized",img_normalized)
    #cv2.waitKey(0)

    # Obtenez le nom du fichier sans le chemin
    file_name = os.path.basename(filename)

    # Construisez le chemin du fichier de sortie
    #output_path = os.path.join('/content/drive/MyDrive/PerceptionAI/data/normalized', file_name)

    # Enregistrez l'image normalisée
    #cv2.imwrite(output_path, img_normalized)

# Close all windows
#cv2.destroyAllWindows()

In [5]:
#first method I try is to retrieve with a single filter knowing that the camera is positioned at 255 centimeters from the ground I get everithing between 170 and 254
depth_map = images_normalized[1][:, :, 2]

# Set the lower and upper thresholds for filtering
lower_threshold = 30
upper_threshold = 180

# Create a binary mask based on the threshold values
binary_mask = cv2.inRange(depth_map, lower_threshold, upper_threshold)

# Apply the binary mask to the original image to extract the filtered pixels
filtered_image = cv2.bitwise_and(depth_map, depth_map, mask=binary_mask)

# Display the original and filtered images
#cv2_imshow(depth_map)
cv2.imshow( "filtered_image",filtered_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#good but I still get the wall
contours, hierarchy  = cv2.findContours(filtered_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour_image = np.copy(images_normalized[0])
for i, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    print(f"Area of Object {i + 1}: {area} square units")

In [ ]:
# Draw contours on the original image
contour_image = np.copy(images_normalized[1])
filtered_contours = [contour for contour in contours if 3000 < cv2.contourArea(contour) < 9000]

cv2.drawContours(contour_image, filtered_contours, -1, (0, 255, 0), 2) 
cv2_imshow(contour_image)

In [ ]:
def show_image(image):
    cv2_imshow(image)
    c = cv2.waitKey()
    if c >= 0 : return -1
    return 0

In [ ]:
for i in range(len(images_normalized)):
  depth_map = images_normalized[i][:, :, 2]

  # Set the lower and upper thresholds for filtering
  lower_threshold = 30
  upper_threshold = 180

  # Create a binary mask based on the threshold values
  binary_mask = cv2.inRange(depth_map, lower_threshold, upper_threshold)

  # Apply the binary mask to the original image to extract the filtered pixels
  filtered_image = cv2.bitwise_and(depth_map, depth_map, mask=binary_mask)
  contours, hierarchy  = cv2.findContours(filtered_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contour_image = np.copy(images_normalized[i])
  #contour_image = np.copy(images_normalized[i])
  filtered_contours = [contour for contour in contours if (3000 < cv2.contourArea(contour) < 12000) and not (5300 < cv2.contourArea(contour) < 5800)]
  for j, filtered_contour in enumerate(filtered_contours):
    area = cv2.contourArea(filtered_contour)
    print(f"Area of Object {j + 1} of image {i}: {area} square units")
  print("------------------------------------------------------------")

  cv2.drawContours(contour_image, filtered_contours, -1, (0, 255, 0), 2) 
  show_image(contour_image)

In [ ]:
# connected components
# getting mask with connectComponents
ret, labels = cv2.connectedComponents(filtered_image)
for label in range(1,ret):
    mask = np.array(labels, dtype=np.uint8)
    mask[labels == label] = 255
    cv2_imshow(mask)
    cv2.waitKey(0)